In [62]:
# 환경변수 설정
from dotenv import load_dotenv
load_dotenv()

True

### Chain을 이용한 Simple LLM

1. PromptTemplate
2. LLM
3. OutputParser

In [63]:
# 1. PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ('system', '너는 포켓몬 전문가야. 사용자의 질문에 친절하고 상세하게 답변해줘.'),
    ('user', '{question}')    
])

In [64]:
# 2. Model
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.3
)

In [65]:
# 3. OutputParser (StrOutputParser)
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [66]:
# 4. Chain 구성
# - 질의 / Question: 어떤 포켓몬이 가장 강해?

chain = prompt_template | model | output_parser
chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

'"가장 강한 포켓몬"이라는 것은 주관적인 질문이며, 이는 여러 요소에 따라 달라질 수 있습니다. 통계적인 측면에서 보면, 포켓몬의 종류에 따라 최대 CP(Combat Power)가 다릅니다. 예를 들어, 아르세우스는 모든 포켓몬 중에서 가장 높은 베이스 스탯을 가지고 있습니다.\n\n하지만, 포켓몬의 강함은 단순히 스탯만으로 결정되지 않습니다. 포켓몬의 유형, 능력, 사용하는 기술, 그리고 트레이너와의 호흡 등도 중요한 요소입니다. 예를 들어, 특정 상황에서는 물 타입의 포켓몬이 불 타입의 포켓몬보다 더 강력할 수 있습니다.\n\n그리고 포켓몬 배틀은 단순히 강함만이 아닌 전략적인 요소도 크게 작용합니다. 적절한 전략과 포켓몬의 조합을 사용하면 상대적으로 약한 포켓몬들로도 강력한 포켓몬을 이길 수 있습니다.\n\n따라서, "가장 강한 포켓몬"을 결정하는 것은 많은 변수들을 고려해야 하며, 개인의 플레이 스타일과 선호도에 크게 의존합니다.'

In [67]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(prompt_template, model, output_parser)
chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

'"가장 강한 포켓몬"이라는 것은 정의하기 어렵습니다. 포켓몬의 강함은 그 포켓몬의 능력치, 사용하는 기술, 트레이너의 전략 등 여러 요소에 따라 달라질 수 있습니다. \n\n그러나 일반적으로, 전설의 포켓몬이나 신화의 포켓몬들은 높은 능력치와 독특한 기술을 가지고 있어 강력한 포켓몬으로 알려져 있습니다. 예를 들어, 아르세우스는 모든 능력치가 120으로 균형있게 높아 많은 사람들이 "가장 강한 포켓몬"으로 여기는 경우가 많습니다. 또한, 레쿠쟈, 디아루가, 파르토라 등의 포켓몬도 높은 능력치를 가지고 있습니다.\n\n그러나 이런 포켓몬들도 상대 포켓몬의 타입이나 기술, 전략에 따라 이길 수도 있고 지울 수도 있습니다. 따라서 포켓몬의 강함은 상대적인 개념이며, 트레이너의 전략과 기술 선택, 포켓몬의 레벨 등 여러 요소가 결합된 결과라고 할 수 있습니다.'

---

### 단계별 Chatbot
- 첫 대화에서 내 이름을 알려주고, 다음 대화에서 내 이름 기억하는지 물어보기!

1. 그냥 Chat
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [68]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.5
)

In [69]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage('안녕! 내 이름은 키티야!')]).content

'안녕, 키티! 만나서 반가워! 어떻게 지내?'

In [70]:
model.invoke([HumanMessage('내 이름이 뭐라고 했지?')]).content

'죄송하지만, 이전 대화 내용을 기억할 수 없어서 당신의 이름을 알 수 없습니다. 당신의 이름을 말씀해 주시면 좋겠습니다!'

2. 직접 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [71]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.5
)

In [72]:
from langchain_core.messages import HumanMessage, AIMessage

first_response = model.invoke([HumanMessage('안녕! 내 이름은 키티야!')]).content

In [73]:
model.invoke([HumanMessage('안녕! 내 이름은 키티야!'),
              AIMessage(first_response),
              HumanMessage('내 이름이 뭐라고 했지?')
]).content

'너의 이름은 키티야! 맞지?'

3. Memory로 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스
- langchain_core.runnables의 클래스
- langchain_core.prompts의 클래스

In [74]:
from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        super().__init__()
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [75]:
store = {} # item(key=session_id, value=InMemoryHistory 인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [76]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [77]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 나무 두 그루를 키우고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 나무를 키우고 있다니 흥미로운 설정이네요! 어떤 종류의 나무를 키우고 있는지, 그리고 그 나무들과 어떤 이야기가 있는지 궁금합니다. 토끼와 나무 사이의 특별한 관계가 있을까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 40, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ6PEPdQQlN69ZOJCuwNsghz7zHog', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--d941e028-9914-4a69-954d-2406d8b32fd8-0' usage_metadata={'input_tokens': 40, 'output_tokens': 59, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [85]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 나무 세 그루를 키우고 있습니다.'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 나무 세 그루를 키우고 있다는 설정이 흥미롭네요! 다람쥐가 나무를 어떻게 돌보는지, 또는 그 나무들이 어떤 역할을 하는지에 대한 이야기를 만들어 볼 수 있을 것 같아요. 예를 들어, 그 나무들은 다람쥐의 집이 되거나, 먹이를 제공하는 나무일 수도 있겠죠. 어떤 이야기로 이어가고 싶으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 140, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ6QCstQSsUAHZL8WSULFaOsPjNn8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--2d0f7bba-458e-4f6c-a39a-797775ac5889-0' usage_metadata={'input_tokens': 140, 'output_tokens': 104, 'total_tokens': 244, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [79]:
store['rabbit']

[HumanMessage(content='토끼는 나무 두 그루를 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 나무를 키우고 있다니 흥미로운 설정이네요! 어떤 종류의 나무를 키우고 있는지, 그리고 그 나무들과 어떤 이야기가 있는지 궁금합니다. 토끼와 나무 사이의 특별한 관계가 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 40, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ6PEPdQQlN69ZOJCuwNsghz7zHog', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d941e028-9914-4a69-954d-2406d8b32fd8-0', usage_metadata={'input_tokens': 40, 'output_tokens': 59, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'aud

In [80]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 나무도 키우지만 당근밭도 하나 가지고 있습니다.'},
    config={'configurable': {'session_id': 'rabbit'}}
)

print(response)

content='토끼가 당근밭도 가지고 있다니 정말 귀엽고 재미있는 설정이네요! 당근밭에서 자라는 신선한 당근은 토끼에게 아주 중요한 음식이겠죠. 나무와 당근밭은 서로 어떤 역할을 할까요? 예를 들어, 나무는 그늘을 제공하거나, 당근은 토끼의 주요 식량이 되는 등 다양한 이야기가 펼쳐질 수 있을 것 같아요. 토끼의 일상이나 그 두 가지를 키우면서 어떤 모험이 있는지 들려줄 수 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 125, 'total_tokens': 248, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CZ6PJekWt180NsBT9zHDJw8lI6bK0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--5f685f0e-62d2-4209-b383-bb439530f687-0' usage_metadata={'input_tokens': 125, 'output_tokens': 123, 'total_tokens': 248, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {

In [86]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼와 다람쥐는 총 몇그루의 나무를 키우고 있나요?'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 나무 세 그루를 키우고 있다고 하셨으니, 토끼가 나무를 키우고 있는지에 대한 정보가 필요합니다. 만약 토끼가 나무를 키우지 않는다면, 총 나무의 수는 3그루입니다. 토끼도 나무를 키우고 있다면, 그 수를 알려주시면 총 몇 그루인지 계산해드릴 수 있습니다!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 275, 'total_tokens': 370, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ6QGBf8BtckYuJu45m1oltiO4U1q', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--296d71bc-facd-4d42-941a-d2f699c9b6bf-0' usage_metadata={'input_tokens': 275, 'output_tokens': 95, 'total_tokens': 370, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [87]:
store

{'rabbit': [HumanMessage(content='토끼는 나무 두 그루를 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='토끼가 나무를 키우고 있다니 흥미로운 설정이네요! 어떤 종류의 나무를 키우고 있는지, 그리고 그 나무들과 어떤 이야기가 있는지 궁금합니다. 토끼와 나무 사이의 특별한 관계가 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 40, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ6PEPdQQlN69ZOJCuwNsghz7zHog', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d941e028-9914-4a69-954d-2406d8b32fd8-0', usage_metadata={'input_tokens': 40, 'output_tokens': 59, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_deta